In [ ]:
import os
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer
from textsummarizer.logging import logger

In [ ]:
%pwd

In [ ]:
%cd ..

In [ ]:
%pwd

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformation:
    root_dir: Path
    data_path: Path
    tokenizer_name: str


    def convert(self):
        try: 
            dataset_samsum = load_from_disk(self.data_path)
            dataset_samsum_pt = dataset_samsum.map(self.preprocess_function, batched = True)
            dataset_samsum_pt.save_to_disk(os.path.join(self.root_dir,"samsum_dataset"))
        except Exception as e:
            raise e

    def preprocess_function(self,example_batch):
        try:
            tokenizer = AutoTokenizer.from_pretrained(self.tokenizer_name)
            input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
            
            with tokenizer.as_target_tokenizer():
                target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
                
            return {
                'input_ids' : input_encodings['input_ids'],
                'attention_mask': input_encodings['attention_mask'],
                'labels': target_encodings['input_ids']
            }
        except Exception as e:
                raise e
    




# def __post_init__(self):
#     self.tokenizer = AutoTokenizer.from_pretrained(self.tokenizer_name)

In [ ]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories


In [ ]:
class ConfigHandler:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        """
        Initialize the ConfigHandler with the provided configuration and parameters file paths.

        Args:
            config_filepath: Path to the configuration file (default: CONFIG_FILE_PATH)
            params_filepath: Path to the parameters file (default: PARAMS_FILE_PATH)

        Returns:
            None

        """
        self.config = self._load_config(config_filepath)  # Load the configuration from the specified file
        self.params = self._load_params(params_filepath)  # Load the parameters from the specified file

        create_directories([self.config.artifacts_root])  # Create necessary directories based on the configuration

    def _load_config(self, config_filepath):
        """
        Load the configuration from the specified YAML file.

        Args:
            config_filepath: Path to the configuration file

        Returns:
            The loaded configuration as a dictionary

        """
        return read_yaml(config_filepath)  # Read and return the configuration from the YAML file

    def _load_params(self, params_filepath):
        """
        Load the parameters from the specified YAML file.

        Args:
            params_filepath: Path to the parameters file

        Returns:
            The loaded parameters as a dictionary

        """
        return read_yaml(params_filepath)  # Read and return the parameters from the YAML file

    
    def get_data_transformation_config(self) -> DataTransformation:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformation(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [ ]:
try:
    config = ConfigHandler()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation_config.convert()
except Exception as e:
    raise e